# ball-tree

In [ ]:
import glob
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits

In [24]:
import numpy as np
import os
from sklearn.neighbors import BallTree
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

####################################################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/dr7_all.fits'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v5/cmatch_10_not_enough.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v5/'
####################################################################################################


def match(X1, X2, max_distance):
    X1 = X1 * (np.pi / 180.)
    X2 = X2 * (np.pi / 180.)
    max_distance = max_distance * (np.pi / 180.)

    # Convert 2D RA/DEC to 3D cartesian coordinates
    Y1 = np.transpose(
        np.vstack([
            np.cos(X1[0]) * np.cos(X1[1]),
            np.sin(X1[0]) * np.cos(X1[1]),
            np.sin(X1[1])
        ]))
    Y2 = np.transpose(
        np.vstack([
            np.cos(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 0]) * np.cos(X2[:, 1]),
            np.sin(X2[:, 1])
        ]))

    max_r = np.sqrt(2 - 2 * np.cos(max_distance))

    tree = BallTree(Y2)
    num = tree.query_radius(Y1, r=max_r, count_only=True)
    ind = tree.query_radius(Y1, r=max_r)
    return num, ind


def r2arcsec(r):
    return r * 180 / np.pi


def max_dis_deg(z):
    '''
    150kpc ~= degree
    '''
    dis = cosmo.comoving_distance(z)  # Mpc
    r = 0.15 / dis.value
    max_r = r2arcsec(r)  # degree
    return max_r


def cmatch():
    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        data = f_drp[1].data
        manga = np.empty((len(data), 2), dtype=np.float64)
        manga[:, 0] = data['objra']
        manga[:, 1] = data['objdec']
        manga_z = data['z']
        plateifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0,len(manga)):
            
            z = manga_z[i]
            pifu = plateifu[i]
            if os.path.exists(dir_res+pifu+'.fits'):
                continue
            else:
                max_r = max_dis_deg(z)
                num,ind = match(manga[i],dr7,max_r)
                array=[]
                names=['ra','dex','z','index']
                if num[0]>0:
                    for j in range(0,num[0]):

                        index=ind[0][j]
                        array.append([dr7[index,0],dr7[index,1],dr7_z[index],index])

                    array=np.array(array)
                    t=Table(array,names=names,dtype=['f8','f8','f8','i4'])
                    t.write(dir_res+pifu+'.fits',format='fits')
                else:
                    with open(dir_res+'no_match.txt','a+') as f:
                        print(plateifu[i],file=f)
            
if __name__=='__main__':
    cmatch()

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astro

# test

In [33]:

import glob
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance, SkyCoord
from astropy.io import fits

file=glob.glob(dir_res+'*fits')
with fits.open('/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits') as f_drp:
    data_drp=f_drp[1].data
    plateifu_drp=data_drp['plateifu']
    ra_drp=data_drp['objra']
    dec_drp=data_drp['objdec']
    z=data_drp['z']
    for filename in file:
        plateifu=filename[len(dir_res):-5]
        print(plateifu)
        index=np.where(plateifu_drp==plateifu)[0]
        ra1=ra_drp[index]
        dec1=dec_drp[index]
        z1=z[index]
        dis=Distance(z=z1).value
        theta=0.15/(dis)
        with fits.open(filename) as f:
            data=f[1].data
            ra=data['ra']
            dec=data['dex']
            if len(ra)==0:
                print('null',plateifu)
            else:
                manga=SkyCoord(ra=ra1*u.deg,dec=dec1*u.deg)
                catalog=SkyCoord(ra=ra*u.deg,dec=dec*u.deg)
                dis=manga.separation(catalog).radian
                dis_max=np.max(dis)

                if dis_max<=theta:
                    continue
                else:
                    print(plateifu)


10001-12701
10001-12702
10001-12703
10001-12704
10001-12704
10001-12705
10001-1901
10001-1902
10001-3701
10001-3702
10001-3702
10001-3703
10001-3704
10001-6101
10001-6102
10001-6102
10001-6103
10001-6103
10001-6104
10001-9101
10001-9102
10141-12701
null 10141-12701
10141-12702
null 10141-12702
10141-12703
null 10141-12703
10141-12704
null 10141-12704
10141-12705
null 10141-12705
10141-1901
null 10141-1901
10141-1902
null 10141-1902
10141-3701
null 10141-3701
10141-3702
null 10141-3702
10141-3703
null 10141-3703
10141-3704
null 10141-3704
10141-6101
null 10141-6101
10141-6102
null 10141-6102
10141-6103
null 10141-6103
10141-6104
null 10141-6104
10141-9101
null 10141-9101
10141-9102
null 10141-9102
10142-12701
null 10142-12701
10142-12702
null 10142-12702
10142-12703
null 10142-12703
10142-12704
null 10142-12704
10142-12705
null 10142-12705
10142-1901
null 10142-1901
10142-1902
null 10142-1902
10142-3701
null 10142-3701
10142-3702
null 10142-3702
10142-3703
null 10142-3703
10142-3704
nul

10517-3702
10517-3703
10517-3704
10517-6101
10517-6102
10517-6103
10517-6104
10517-9101
10517-9102
10519-12701
10519-12702
10519-12703
10519-12704
10519-12705
10519-1901
10519-1902
10519-3701
10519-3702
10519-3703
10519-3704
10519-6101
10519-6102
10519-6103
10519-6104
10519-9101
10519-9102
7443-12701
7443-12701
7443-12702
7443-12703
7443-12704
7443-12705
7443-1901
7443-1902
7443-1902
7443-3701
7443-3702
7443-3703
7443-3704
7443-6101
7443-6102
7443-6102
7443-6103
7443-6104
7443-9101
7443-9101
7443-9102
7495-12701
7495-12702
7495-12702
7495-12703
7495-12704
7495-12704
7495-12705
7495-1901
7495-1902
7495-3701
7495-3702
7495-3703
7495-3704
7495-3704
7495-6101
7495-6102
7495-6103
7495-6103
7495-6104
7495-9101
7495-9102
7815-12701
7815-12702
7815-12703
7815-12704
7815-12705
7815-1901
7815-1902
7815-3701
7815-3702
7815-3703
7815-3704
7815-6101
7815-6102
7815-6103
7815-9101
7815-9102
7957-12701
7957-12702
7957-12702
7957-12703
7957-12704
7957-12705
7957-1901
7957-1902
7957-1902
7957-3701
7957-

8140-3703
8140-3704
8140-6101
8140-6101
8140-6102
8140-6103
8140-6104
8140-9101
8140-9102
8141-12701
8141-12701
8141-12702
8141-12703
8141-12704
8141-12705
8141-1901
8141-1902
8141-3701
8141-3702
8141-3702
8141-3703
8141-3704
8141-6101
8141-6102
8141-6103
8141-6103
8141-6104
8141-9101
8141-9102
8143-12701
8143-12702
8143-12703
8143-12704
8143-12705
8143-1901
8143-1902
8143-1902
8143-3701
8143-3701
8143-3702
8143-3703
8143-3704
8143-6101
8143-6102
8143-6103
8143-6104
8143-9101
8143-9102
8144-12701
8144-12702
8144-12702
8144-12703
8144-12704
8144-1901
8144-1902
8144-1902
8144-3701
8144-3702
8144-3703
8144-3704
8144-6101
8144-6102
8144-6103
8144-6104
8144-9101
8144-9101
8144-9102
8145-12701
8145-12702
8145-12703
8145-12704
8145-12705
8145-1901
8145-1901
8145-1902
8145-3701
8145-3701
8145-3702
8145-3702
8145-3703
8145-3704
8145-6101
8145-6102
8145-6103
8145-6104
8145-9101
8145-9102
8146-12701
8146-12701
8146-12702
8146-12703
8146-12704
8146-12704
8146-12705
8146-1901
8146-1902
8146-3701
81

ValueError: Distance must be >= 0.  Use the argument 'allow_negative=True' to allow negative values.

In [28]:
from astropy.coordinates import Distance

dis=Distance(z=0.03)
dis.value

135.79570049977337